<a href="https://colab.research.google.com/github/AsimZz/Univeristy-of-Khartoum-Graduation-Project/blob/master/cnn_models_and_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initial Setup

In [ ]:
!pip install torch==1.6.0 torchvision==0.7.0

In [ ]:
import torch
import numpy as np 
from tqdm import tqdm

%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# Preprocessing Setup

In [ ]:
from device import get_default_device
device = get_default_device()
device

In [ ]:
data_origin = "/path/to/the/dataset/"

In [ ]:
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from helpers import tiff_loader

data_transforms = transforms.Compose([transforms.ToTensor()])
image_datasets = ImageFolder(root=data_origin, loader=tiff_loader)

In [ ]:
from dataset import EuroSatDataset

## run this cell only once if you didn't save the EuroDataset class object
images = []
labels = []

image_datasets = EuroSatDataset(images, labels)

for image, label in tqdm(image_datasets):
    images.append(image)
    labels.append(labels)

In [ ]:
import pickle

save_path = '/path/to/save/the/files'

In [ ]:
from helpers import load_dataset_from_pickle, save_dataset_in_pickle

## then we save the model in drive

save_dataset_in_pickle(save_path, image_datasets)
image_datasets = load_dataset_from_pickle(save_path)

In [ ]:
len(image_datasets)

In [ ]:
from helpers import split_dataset

train_data, val_data, test_data = split_dataset(image_datasets)

In [ ]:
print("Train Length = " + str(len(train_data)))
print("Validation Length = " + str(len(val_data)))
print("Test Length = " + str(len(test_data)))

In [ ]:
from torch.utils.data import DataLoader

batch_size = 64
kw = {"num_workers": 8, "pin_memory": True} if device == "cuda" else {}

train_loaders = DataLoader(train_data, batch_size=batch_size, shuffle=True, **kw)
val_loaders = DataLoader(val_data, batch_size=batch_size, shuffle=True, **kw)
test_loaders = DataLoader(test_data, batch_size=batch_size, shuffle=True, **kw)

In [ ]:
from device import DeviceDataLoader

train_loader = DeviceDataLoader(train_loader, device)
val_loader = DeviceDataLoader(val_loader, device)

In [ ]:
torch.cuda.empty_cache()

# Model Architectures

In [ ]:
from models import ResNet152, GoogleNet, EfficientNet
from trainer import ModelTrainer
from device import to_device

resnet_model = ResNet152(13, 10)
googleNet_model = GoogleNet(13, 10)
enet_model = EfficientNet(
    version="b0",
    num_classes=10,
)

## ResNet-152 Model

Residual network use the concept of residual blocks. it was proved that ResNet can perform very well in image classification tasks.
The architecture of the network is explained in the figure below:

![alt text](https://www.researchgate.net/profile/Dongyun-Lin/publication/324961229/figure/fig2/AS:633700479954944@1528097376059/The-basic-architecture-of-Resnet152.png)

### Training The ResNet model

In [ ]:
resnet_model = to_device(resnet_model, device)
resnet_trainer = ModelTrainer(resnet_model, "resnet", train_loader, val_loader)
history = [resnet_trainer.evaluate()]


# history += resnet_trainer.train(epochs=20,save_path="",)

# The path where you saved the model weights
saved_model = torch.load(save_path)

saved_resnet_model = to_device(ResNet152(13, 10), device)
saved_resnet_model.load_state_dict(saved_model["resnet"])

check_accuracy(train_loader, resnet_model)

## GoogleNet (InceptionNet) Model

<img src="https://www.researchgate.net/profile/Bo-Zhao-67/publication/312515254/figure/fig3/AS:489373281067012@1493687090916/nception-module-of-GoogLeNet-This-figure-is-from-the-original-paper-10.png" width=500>

### Training The GoogleNet model

In [ ]:
googleNet_model = to_device(googleNet_model, device)
googleTrainer = ModelTrainer(googleNet_model, "resnet", train_loader, val_loader)
history = [googleTrainer.evaluate()]


# history += googleTrainer.train(epochs=20,save_path="")

saved_model = torch.load(save_path)

saved_googleNet_model = to_device(GoogleNet(13, 10), device)
saved_googleNet_model.load_state_dict(saved_model["googleNet_model"])

check_accuracy(train_loader, googleNet_model)

## EfficientNet Model


### Training The EffNet model

In [ ]:
efficentnet_model = to_device(enet_model, device)
efficentnet_trainer = ModelTrainer(
    resnet_model, "efficentnet", train_loader, val_loader
)
history = [efficentnet_trainer.evaluate()]

# history += efficentnet_trainer.train(epochs=20,save_path="")


saved_model = torch.load(save_path)

saved_effNet_model = to_device(EfficientNet("b0", 10), device)
saved_effNet_model.load_state_dict(saved_model["effNet_model"])

check_accuracy(train_loader, saved_effNet_model)

# Other Metrics

In [ ]:
from metrics import check_metric
from sklearn import metrics

### Precision, Recall and F1 Score For GoogleNet

In [ ]:
check_metric(test_loaders,saved_googleNet_model,metrics.recall_score,'macro','Recall')

In [ ]:
check_metric(test_loaders,saved_googleNet_model,metrics.precision_score,'macro','Precision')

In [ ]:
check_metric(test_loaders,saved_googleNet_model,metrics.f1_score,'macro','F1')

### Precision, Recall and F1 score for ResNet152

In [ ]:
check_metric(test_loaders,saved_resnet_model,metrics.recall_score,'macro','Recall')

In [ ]:
check_metric(test_loaders,saved_resnet_model,metrics.precision_score,'macro','Precision')

In [ ]:
check_metric(test_loaders,saved_resnet_model,metrics.f1_score,'macro','F1')

### Precision, Recall and F1 score for EfficientNet

In [ ]:
check_metric(test_loaders,saved_effNet_model,metrics.recall_score,'macro','Recall')

In [ ]:
check_metric(test_loaders,saved_effNet_model,metrics.precision_score,'macro','Precision')

In [ ]:
check_metric(test_loaders,saved_effNet_model,metrics.f1_score,'macro','F1')